# J5実験 第3回

第2回で，
- 電卓
- 単純なプログラミング言語のインタプリタ（解釈実行器）

を作り，自分でプログラミング言語に構文を追加し，Lex・Yacc・Cを使ってプログラミング言語処理系を作成する方法が分かってきたと思います．
今回はその知識を利用して，

- 簡単な英文処理プログラム
- 単純なプログラミング言語のコンパイラ

を作成してみましょう．

忘れてしまったところは，過去の授業資料 1.ipynb と 2.ipynb を参照してください．

# 簡単な英文処理プログラム

Lex や Yacc はもともと「コンパイラを簡単に作るためのツール」として作られたものです．
とはいえ，字句のパターンと構文規則を与えれば，テキストを解釈できるプログラムを作ることができるので，色々な応用が考えられます．

そういった応用例の1つとして，かなりいいかげんなものですが，簡単な英文の解析プログラムをここでは作ってみましょう．

与えられた英文テキストに対して，英文法の構成要素を解析し，テキストに情報を付加します．こういった品詞などの解析は， **[形態素解析](https://ja.wikipedia.org/wiki/%E5%BD%A2%E6%85%8B%E7%B4%A0%E8%A7%A3%E6%9E%90)** と呼ばれています．
自然言語処理（「プログラミング言語」と区別するために，情報の分野ではそう呼ばれる）では必要な処理ですので，ここまでのプログラミング言語を作るというところに興味がない・他人事だと思っていた人も，**テキストを読み込むプログラムを作る**ということの必要性が分かるのではないでしょうか．

（ここでやるのは，かなりいいかげんな解析ですが）雰囲気を見てみましょう．

まず，Lexを以下のように定義します．

In [ ]:
/* lex ex3-1.lex */
alpha   [a-zA-Z]
white   [\t ]
%%
CATS|DOGS|MAN|WOMAN         { strcpy(yylval.name, yytext); return NOUN; }

ROOM|HOUSE|SKY              { strcpy(yylval.name, yytext); return PLACE_NOUN; }

THE|A|AN|THIS|THAT|THOSE|MY { strcpy(yylval.name, yytext); return DETERMINER; }

FIRST|LAST|NEXT|SMALL|BIG   { strcpy(yylval.name, yytext); return ADJECTIVE; }

IS|ARE|RUN|FLY|FIND|BE      { strcpy(yylval.name, yytext); return VERB; }

CAN|SHOULD|MAY              { strcpy(yylval.name, yytext); return AUXILIARY_VERB; }

IN|AT|ON|TO|INSIDE          { strcpy(yylval.name, yytext); return PREPOSITIONAL; }

[\.,']                      { return yytext[0]; }
{white}                     { ; }

今は，考えやすくするため，使える英単語をここに書かれたものに限定します．
つまり，

> THE MAN MAY FIND THOSE DOGS AT THAT HOUSE.

のような構文を解析できる解析器ということですね．
アクションはこれまで出てきた Lex のプログラムと同等ですが，**Lex で `yylval` に入れておいた値は，Yaccの属性値として利用することができる** という使い方は新たに出てきたものです．
こうすることで，Yaccのアクションで直接`$1`などの値を読み出せるので，定義が簡潔になります．

それでは，Yaccのプログラムを見てみましょう．

In [ ]:
/* yacc ex3-1.yacc */
%union {
  char   name[128];
}
%token <name> PREPOSITIONAL VERB AUXILIARY_VERB PLACE_NOUN NOUN DETERMINER ADJECTIVE

%type  <name> np det vp pp obj
%%
ss : np vp '.'  { printf("NP = %s\nVP = %s\n", $1, $2); }
   ;

np : DETERMINER NOUN         { sprintf($$,"%s %s", $1, $2); }
   | ADJECTIVE det NOUN      { sprintf($$, "%s %s %s", $1, $2, $3); }
   | NOUN                    { strcpy($$, $1); }
   ;

det : DETERMINER   { strcpy($$, $1); }
    |              { strcpy($$, ""); }
    ;

vp : VERB             { strcpy($$, $1); strcat($$," "); }
   | AUXILIARY_VERB VERB obj { sprintf($$, "%s %s %s", $1, $2, $3); }
   | VERB det NOUN pp        { sprintf($$, "%s %s %s %s", $1, $2, $3, $4); }
   ;

pp : PREPOSITIONAL det PLACE_NOUN    { sprintf($$, "%s %s %s", $1, $2, $3); }
   |                                 { strcpy($$, ""); }
   ;

obj : det NOUN pp   { sprintf($$, "%s %s %s", $1, $2, $3); }
    | PREPOSITIONAL det PLACE_NOUN    { sprintf($$, "%s %s %s", $1, $2, $3); }
    |               { sprintf($$, ""); }
    ;

文は `NP VP` （名詞句と動詞句）です．ここに構文規則を追加していけば，解析できるパターンが増えていくのは，ここまでのYaccの知識で理解できると思います．Lexで `yylval.name` に値を入れておいたことにより，`det : DETERMINER   { strcpy($$, $1); }` のように `token` の値を属性値`$1`として使うことができます．

Cプログラムは，これまでのプログラムのままです．

In [ ]:
/* c ex3-1.c */
#include <stdio.h>
extern char *yytext;

int value;

#include "y.tab.c"
#include "lex.yy.c"

int main() {
  yyparse();
  return 0;
}

それでは，上で例に挙げた文を解析してみましょう．

In [ ]:
/* a.out */
THE MAN MAY FIND THOSE DOGS AT THAT HOUSE.

同様に，別の文でも解析できます．

In [ ]:
/* a.out */
CATS CAN FLY TO THE SKY.

ここでは，「文」に対する解析結果だけを出力していますが，たとえば `VP` や `PP` がより細かくどういう品詞に分解されるかなども構文定義では分けてありますので，アクションを記述すれば可能だと分かるはずです．さらに，第2回でプログラミング言語に対して作った抽象構文木のような中間構造を用いれば，解析結果を色々な用途で利用できます．
実際には，**世の中にある英語の文にマッチするような構文を用意できるわけではなく**（言語学の分野では統語論などとして知られるが，たぶん一般的な構文ルールを用意するのは無理なんじゃないかなぁ...（個人の感想です）），実用されている形態素解析はこれほど簡単に解析できるものではないです．そのため，構文以外の様々な手法を用いて解析する必要があます（興味がある人は調べてみてください）．ここでは，プログラミング言語の生成以外のLexやYaccの使用例を紹介しました．

以上のように，英文に限らず，コマンドの設定ファイルや，データ形式（たとえば，JSON形式）を読み込みたいというときにも，テキストを読み込んで意味を解釈するプログラムの必要性が理解できたと思います．

# 単純なプログラミング言語のコンパイラ

第2回までで，中間形式（抽象構文木）の作成と，その解釈実行のプログラム（インタプリタ）を作成できました．

ここではコンパイラを作ってみましょう．
コンパイラは，**ソースコードを別のプログラミング言語のプログラムに翻訳する**プログラムです．翻訳先の言語は，
- より低級な言語であったり，
- 実行しやすい（便利な）言語であったり

して，状況に応じて翻訳先を考えます．

すでに，ソースコードの木構造を作るところまでは前回までできていますので，Cプログラム中の`dotree()`で実行する際にコード生成をすればよいわけです．

## コード生成 - C言語へのコンパイル

まずは，C言語へのコンパイルを考えてみましょう．
C言語のコンパイラは様々実装されていますし，色々な環境で動かせるコードを吐き出せるので，自分が作った言語のコンパイル先としてC言語を考えることはありえます（上の翻訳先の言語の箇条書きでいう「実行しやすい便利な言語」ということです）．また，みなさんもC言語は読めるはずですし，第2回までで作ってきた言語もC言語から大きく離れた構文ではないため，コンパイル先として難しくありません．

ここでは，`dotree()` のかわりに，`emit_c()` を定義して，コンパイルの例を考えてみましょう．

簡単のため，前回の `while`文が入る前の Lex と Yacc をそのまま使います．

In [ ]:
/* lex ex3-2.lex */
alpha   [a-zA-Z]
digit   [0-9]
white   [\n\t ]
%%
read                        { return READ; }
print                       { return PRINT; }
{alpha}({alpha}|{digit})*   { return IDENT; }
{digit}+                    { return NUM; }
[+\-=();{}]                 { return yytext[0]; }
{white}                     { ; }

In [ ]:
/* yacc ex3-2.yacc */
%union {
  Node* np;
  int i;
}
%type <np> stlist stat expr prim var
%token NUM;
%token IDENT;
%token READ;
%token PRINT;
%%
prog   : IDENT '{' stlist '}'       { emit_c($3); return 0; }
       ;
stlist :                            { $$ = NULL; }
       | stlist stat                { $$ = createNode(T_STLIST, $1, $2); }
       ;
stat   : var '=' expr ';'           { $$ = createNode(T_ASSIGN, $1, $3); }
       | READ var ';'               { $$ = createNode(T_READ, $2, NULL); }
       | PRINT expr ';'             { $$ = createNode(T_PRINT, $2, NULL); }
       ;
expr   : prim              { $$ = $1; }
       | expr '+' prim     { $$ = createNode(T_ADD, $1, $3); }
       | expr '-' prim     { $$ = createNode(T_SUB, $1, $3); }
       ;
prim   : NUM               { $$ = createNode(T_NUM, atoi(yytext), NULL); }
       | var               { $$ = createNode(T_VAR, $1, NULL); }
       | '(' expr ')'      { $$ = $2; }
       ;
var    : IDENT             { $$ = lookup(yytext); }
       ;

Cへコンパイルするには，今作成している言語との差分を考慮しなくてはいけないので，以下のCプログラムでのポイントは，

- Cの`main`関数をちゃんと書く
- 変数宣言をする
- Cの関数を呼び出すように変換する

という点です．特に今の言語では，`print`は `printf`へ，`read` は `scanf` へ変換できる1対1の対応があるので，コンパイルしやすいですね．

木をたどるプログラムはそのままで，ノードごとの想定している動作にしたがって，Cのプログラムを出力すればよいので，たとえば，
`T_ASSIGN` のノードであれば，

---
```
case T_ASSIGN:
      printf("\tv%d = ", node->left);
      _emit_c(node->right);
      printf(";\n");
      break;
```
---

のようにして，`変数 = 式;` というCのプログラムにすればよいですね．（`_emit_c`を再帰的に呼べば，式を出力できるように作ってあるため．）
変数のところは，変数表での番号`n`に置き換えて `vn` と出力します．（`T_ASSIGN`のノードで，左の子にそのまま変数番号が入っていたことを思い出してください）

他の木のノードに対しても記述を追加したCプログラムは以下のとおりです．
理解した上で実行してください．

In [ ]:
/* c ex3-2.c */
#include <stdio.h>
struct stab {
  int val;
  char name[20];
} stab[100];
int stabuse = 0;

extern char *yytext;

#define T_STLIST 1
#define T_ASSIGN 2
#define T_READ   3
#define T_PRINT  4
#define T_ADD    5
#define T_SUB    6
#define T_NUM    7
#define T_VAR    8

typedef struct Node {
    int node_type;
    struct Node* left;
    struct Node* right;
} Node;


// 引数にとる文字列が表にあれば見つかった位置を返して，なければ追加する関数
int lookup(char *s) {
  int i;
  // 表の使っているところを順に見ていって
  for (i = 0; i < stabuse; ++i) {
    if (strcmp(stab[i].name, s) == 0) {
      // 見つかった場合はその位置を返す
      return i;
    }
  }
  if (stabuse >= 99) {
    // 変数の上限の個数を超えてしまった
    printf("table overflow,\n");
    exit(1);
  }

  // 表になかったので追加する
  strcpy(stab[stabuse].name, s);
  return stabuse++;  // 追加した位置を返す
}

// 新しいノードを作る関数
Node* createNode(int node_type, Node* left, Node* right) {
    Node* newNode = (Node*)malloc(sizeof(Node));
    newNode->node_type = node_type;
    newNode->left = left;
    newNode->right = right;
    return newNode;
}

void _emit_c(Node* node) {
  if (node == NULL) {
    return;
  }
  switch (node->node_type) {
    case T_STLIST:
      if (node->left != NULL) { _emit_c(node->left); }
      _emit_c(node->right);
      break;
    case T_ASSIGN:
      printf("\tv%d = ", node->left);
      _emit_c(node->right);
      printf(";\n");
      break;
    case T_READ:
      printf("\tscanf(\"%%d\", &v%d);\n", node->left);
      break;
    case T_PRINT:
      printf("\tprintf(\"%%d\",");
      _emit_c(node->left);
      printf(");\n");
      break;
    case T_ADD:
      _emit_c(node->left);
      printf(" + ");
      _emit_c(node->right);
      break;
    case T_SUB:
      _emit_c(node->left);
      printf(" - ");
      _emit_c(node->right);
      break;
    case T_NUM:
      printf("%d", node->left);
      break;
    case T_VAR:
      printf("v%d", node->left);
      break;
  }
}

void emit_c(Node* node) {
  printf("#include <stdio.h>\n");
  printf("int main() {\n");
  int i;
  for (i = 0; i < stabuse; i++) {
    printf("\tint v%d;\n", i);
  }
  _emit_c(node);
  printf("}\n");
}

#include "y.tab.c"
#include "lex.yy.c"

int main() {
  yyparse();
  return 0;
}

それでは，自分達の言語からC言語へコンパイルしてみましょう．

In [ ]:
/* a.out */
main {
  read x;
  y = x + 1;
  print y;
}

a.outの出力結果は，ちゃんとC言語のプログラムになっているはずです．上のポイントとして挙げた箇条書きに対応した形で書くと，
- `emit_c()`の先頭で `main`関数などC言語に必要なものを出力する
- 変数表をたどって，変数宣言をする（`v?`）というCの変数名とする
- 必要なところはCの構文に合わせて，関数などを呼ぶようにする

という点ができていることを確認してください．

さらに，このプログラムをCコンパイラで機械語に翻訳すれば，実行できますので，各自でコピーして実行してみてください．

以上のように，中間構造ができているものに対して，コード生成することに難しいことはありません．

## コード生成 - アセンブリ言語へのコンパイル

次に，アセンブリ言語へのコンパイルを考えてみましょう．
ここでは，**x64アセンブリ言語**への出力を考えます．
アセンブリ言語全般については，もしかしたら，計算機通論の**MIPSアセンブリ言語**を習っているかもしれないので，それを思い出してください．

C言語よりも低級な言語であるので，実行形式を考えつつ，コンパイル（要するにコードの出力）をしないといけません．
x64アセンブリ言語は**レジスタマシン**なので，先のC言語のように1対1で出力するというわけにはいきません．

この実験では x64 のアセンブリ言語の理解が主眼ではないので，実験に必要なところだけ，以下にまとめました．
以下の「x64アセンブリ言語の説明」を広げて読んでください．

さらなる詳細を知りたい場合には，自分で調べてみてください．

<details>
<summary>
x64アセンブリ言語の説明
</summary>
    
---

x64アーキテクチャ（x86_64，amd64とも言う）は，Intel 80386からはじまる IA32（32ビットアーキテクチャ）を64ビットに拡張したものです（IA64という64ビットのアーキテクチャもあるが，それは別ものなので注意）．
    
x64には，64ビット長のレジスタとして，`rax`，`rbx`，`rcx`，`rdx`，`rdi`，`rsi`，`rbp`，`rsp`（以上の8個は，IA32の64ビット拡張版．32ビット長のレジスタ名は`r`の代わりに`e`で始まる），`r8`〜`r15`の16個があります．
gccでの使い方は，以下のように決まっています（整数型/ポインタ型の場合）．

| レジスタ名 | 用途 |
|:-----------|:------------|
| `rax`    | 浮動小数点数レジスタ使用の有無/戻り値 |
| `rdi`    | 1番目の整数型/ポインタ型引数 |
| `rsi`    | 2番目の整数型/ポインタ型引数 |
| `rdx`    | 3番目の整数型/ポインタ型引数 |
| `rcx`    | 4番目の整数型/ポインタ型引数 |
| `r8`     | 5番目の整数型/ポインタ型引数 |
| `r9`     | 6番目の整数型/ポインタ型引数 |
| `rbp`    | ベースポインタ              |
| `rsp`    | スタックポインタ            |

引数が7個以上あると，7番目以降の引数はスタックを使用して渡しますが，x86の場合よりも煩雑なので，ここでは割愛します．浮動小数点数も（別種の）レジスタやスタックを使いますが，これらについても割愛します．なお，呼ばれた関数側で `r12`〜`r15`，`rbx`，`rbp`，`rsp`のレジスタを使用する場合には，あらかじめそれらの値を保存し，関数から戻る前に元に戻す必要があります．

`rax`は，関数の戻り値を保持しますが，関数呼び出し時には浮動小数点数レジスタを用いて引数を渡しているか否かを表すためにも使われます．引数として浮動小数点数を受けとる可能性がある関数を呼び出す場合には，適切な値を`rax`に設定する必要があります．なお，J5実験では，浮動小数点数を扱っていないので，`rax`には常に0を設定します．

アセンブリコードは，基本的には1行1文で書きます．

```
    [ラベル:]   命令   [オペランド1, オペランド2, ...]
```



「ラベル:」を書くと，直後の命令（がメモリ上に置かれたときのメモリ）のアドレスを「ラベル」で参照できるようになります（「ラベル:」単独でも構いません）．ラベルは，（「`.`」を含む）英字に英数字が続いたものです．ラベルは，アドレスを参照するためのものですから，ラベルをいくつ定義しようとも，実行ファイルの大きさには影響しません．

「命令」には2種類があります．

- オペコード - CPUが直接実行できる機械語を表す名前

  `mov`（転送），`add`（加算），`sub`（減算）等の命令では，オペランドの大きさ（バイトサイズ）に応じて，1バイト（8ビット）の場合には「`b`」，2バイト（16ビット）の場合には「`w`」，4バイト（32ビット）の場合には「`l`」，8バイト（64ビット）の場合には「`q`」を付加して明示できます．オペランドにレジスタを指定する場合には，レジスタ名の前に「`%`」を付けます．「数値」を定数としてオペランドに指定するときには，「`$`」を付けて10進表記します（Cのように16進（`0x`），8進（`0`）も使える）．レジスタでも数値定数でないものは，メモリアドレスを指定すると解釈され（もちろん，正しい形式であることが必要），そのアドレスが指すメモリ市のアドレス/内容が操作の対象になります．分岐命令以外では，ラベルのアドレスそのものを参照したい場合には，「`$`」を付ける必要があります．

  算術演算命令では，命令実行後，コンディションコードレジスタに演算結果の符号や0かどうかが設定されます．コンディションコードレジスタの内容は，条件付き分岐命令で（暗黙に）使用します．

- ディレクティブ - メモリ領域の確保，プログラム内で用いる「文字列」などの定数で初期化したメモリ領域，リンカーやデバッガに対する付加情報の指定

  オペコードと区別しやすいようにディレクティブは「`.`」で始まります．


オペコードには，必要な個数のオペランドを「`,`」で区切ってつづけます．（gccが使う）GAS（GNU Assembler）では，オペランドが2つの場合には，

```
    オペランド2 ← オペランド2 <操作> オペランド1
```

となります．代表的な命令は以下のとおりです．
ただし，乗算命令（`imul`）はオペランドが複数の場合もありますが，ここでは1つの場合の説明をしています．


| 種類 | コード | 説明 |
|:----------------|:------------|:------------|
| 転送命令    | `mov src, dst` | `dst ← src` |
|             | `push src`   | `src をプッシュ; mem[--rsp] ← src` |
|             | `pop dst`    | `dst へポップ; dst ← mem[rsp++]` |
|             | `lea src, dsc` | `src`のメモリアドレスそのものを `dst` に入れる |
| 比較命令    | `cmp src, dst` | `dst - src` の結果をコンディションコードレジスタに反映．<br> コンディションコードの「見方」によって，符号付き/符号無しの比較になる． |
| 算術命令    | `add src, dst` | `dst ← dst + src` |
|            | `sub src, dst` |  `dst ← dst - src` |
|             | `imul src`   | `rdx:rax ← rax * src` :  符号付き64ビット同士を乗算する．<br> 結果は符号付き128ビット（上位64ビットは`rdx`レジスタ，下位64ビットは`rax`レジスタ）．<br> 暗黙のうちに，`rax`レジスタを使用することに注意．<br> また，演算結果も，`rax`レジスタと`rdx`レジスタに残る．|
|             | `cqto`      | `rdx ← rax を符号拡張`  : `rax`レジスタ（符号付き64ビット）の符号ビット（0または1）で`rdx`レジスタを満たす． <br>（つまり，各ビットは0または1） | 
|             | `idiv src`  | `rax ← rdx:rax / src; rdx ← rdx:rax` : <br> 符号付き128ビット（上位64ビットは`rdx`レジスタ，下位64ビットは`rax`レジスタ）を`src`（符号付き64ビット）で除算する． <br> 結果（商と余り）は符号付き64ビット．暗黙のうちに，`rax`レジスタと`rdx`レジスタを使用するため，`src`にこれらのレジスタを指定すると予期しない結果となる．<br> なお，演算結果は `rax` レジスタと `rdx` レジスタに残る．呼び出し前に `rdx` レジスタを初期化（0を代入）する必要があることに注意． |
| 分岐命令     | `jmp ラベル`  | 無条件に `ラベル` へ分岐 |
|              | `je ラベル`  | 等しければ `ラベル` へ分岐 |
|              | `jne ラベル`  | 等しくなければ `ラベル` へ分岐 |
|              | `jg ラベル`  | （符号付きで）大きければ `ラベル` へ分岐 |
|              | `jge ラベル`  | （符号付きで）大きいか等しければ `ラベル` へ分岐 |
|              | `jl ラベル`  | （符号付きで）小さけければ `ラベル` へ分岐 |
|              | `jle ラベル`  | （符号付きで）小さいか等しければ `ラベル` へ分岐 |


具体例を見てみましょう．（gccでは，int型は4バイト（32ビット），long型は8バイト（64ビット），ポインタ型は8バイト（64ビット）を占めることに注意）

---

```
int func(int a, int b, int c) {
  extern int func2(int, int);
  return func2(a, b + c);
}
```

---

この簡単なCプログラムをgccにオプション「`-S`」を指定して「`.s`」で終わるファイル名にアセンブリコードを出力できます．

```
% cc -S test1.c
% cat -n test1.s
```

結果の test1.s は以下のようになります．

---

```
    .file "test1.c"
    .text
    .globl func
    .type  func, @function
func:
.LFB0:
    .cfi_startproc
    pushq %rbp
    .cfi_def_cfa_offset 16
    .cfi_offset 6, -16
    movq  %rsp, %rbp
    .cfi_def_cfa_register 6
    subq  $16, %rsp
    movl %edi, -4(%rbp)
    movl %esi, -8(%rbp)
    movl %edx, -12(%rbp)
    movl -12(%rbp), %eax
    movl -8(%rbp), %edx
    addl %eax, %edx
    movl -4(%rbp), %eax
    movl %edx, %esi
    movl %eax, %edi
    call func2
    leave
    .cfi_def_cfa 7, 8
    ret
    .cfi_endproc
.LFE0:
    .size func, .-func
    .ident "GCC: (GNU) 4.8.5 20150623 (RedHat 4.8.5-16)"
    .section  .note.GNU-stack,"",@progbits
```

---

「`.`」（ドット）ではじまる小文字の名前（`.text`や`.type`等）は「ディレクティブ」です．一部は，デバッガ等に対する付加情報等を指定するディレクティブなので，気にする必要はありません．それ以外の箇所は以下のとおりです．

- 2行目 : （命令列を置く）テキストエリアの開始を指定（必須）
- 3行目 : ラベル「`func`」をグローバル名とする指示（省略可）
- 5行目 : ラベル「`func`」の定義位置（`func`の番地が決まる）
- 8〜12行目 : 関数の入口でのスタックポインタ/ベースポインタの保存
- 13行目 : 関数内で使う（引数を含む）変数用の領域をスタックに確保
- 14〜23行目 : 関数の本体に対応するアセンブリコード
- 24行目と26行目 : 関数から戻る処理

関数の入口での「おきまり」の処理（上のプログラムでの8，11，13行目にあたります）

---
```
    pushq %rbp
    movq  %rsp, %rbp
    subq  $16, %rsp
```

---

の意味を見ていきましょう．関数が呼ばれた直後（``pushqを実行する前）には，スタックは以下の図の (a) のようになっていて，スタックトップには「戻りアドレス」が載っています（メモリのアドレスは図の上から下に向かって増加する）．

![スタックの利用方法](https://www.clas.kitasato-u.ac.jp/~takano/uec-lexyacc/stack.png)

「`push %rbp`」を実行すると，(b) のようになり，レジスタ`rbp`（ベースポインタ）を保存します．ついで「`movq %rsp, %rbp`」を実行すると (c) のようになります．最後に「`subq $n, %rsp`」を実行して，レジスタ`rsp`(スタックポインタ）から「`$n`」だけ差し引いて関数内の（引数を含む）ローカル変数のためのメモリ領域を確保します (d)．ここで「`$n`」は，（ローカル変数の個数）× 8 を 16の倍数に切り上げた値です．（正確には，ローカル変数1つあたり一律に8バイトではなく，変数の型に応じたバイト数の総計）．これは，`call`命令を実行するときに（x64アーキテクチャ上）スタックポインタの値が16の倍数となっている必要があるからです．

関数の出口の「おきまり」の処理（上のプログラムでの24行目と26行目にあたります）

---

```
    leave
    ret
```

---

では，命令`leave` によって，レジスタ`rbp`（ベースポインタ）の値をレジスタ`rsp`（スタックポインタ）に入れ（上図の (c) の状態），スタックからレジスタ `rbp`（ベースレジスタにポップして関数呼び出し直後の値に戻します（上図の (a) の状態）．ついで命令 `ret` を実行すると，スタックから戻りアドレスをポップしてそのアドレスへ分岐します（これで呼び出し側に戻る）．

この関数の実行中，レジスタ `%rsp`（スタックポインタ）は命令実行にしたがって変化しますが，レジスタ `rbp`（ベースポインタ）の値を変更してはいけません．`rbp`を基準にして，関数内の（引数を含む）ローカル変数を置いたメモリ位置を参照するとともに，呼び出し前の `rbp` を取り出すスタック位置を示しているからです．

上のプログラムの本体部分の意味は次のとおりです．（int型の変数（なので4バイト長）を指定しているため，各引数を保存するスタック内の場所は，`rbp`（ベースポインタ）を基準にして，第1引数は `-4(%rbp)`，第2引数は `-8(%rbp)`，第3引数は `-12`(%rbp)` となる．）

---

```
    movl %edi, -4(%rbp)      第1引数(int a) の値を保存
    movl %esi, -8(%rbp)      第2引数(int b) の値を保存
    movl %edx, -12(%rbp)     第3引数(int c) の値を保存
    movl -12(%rbp), %eax     %eax ← c
    movl -8(%rbp), %edx      %edx ← b
    addl %eax, %edx          %edx ← b+c
    movl -4(%rbp), %eax      %eax ← a
    movl %edx, %esi          func2 の第2引数 ← %edx （これは b+c）
    movl %eax, %edi          func2 の第1引数 ← %eax （これは a）
    call func2               func2を呼び出す
                             結果は%eaxに入って戻ってくるので，関数funcのとしてそのまま使う
```

---

---
</details>

それでは，C言語にコンパイルしたときと同様の簡単な言語の例で見てみましょう．

In [ ]:
/* lex ex3-3.lex */
alpha   [a-zA-Z]
digit   [0-9]
white   [\n\t ]
%%
read                        { return READ; }
print                       { return PRINT; }
{alpha}({alpha}|{digit})*   { return IDENT; }
{digit}+                    { return NUM; }
[+\-=();{}]                 { return yytext[0]; }
{white}                     { ; }

In [ ]:
/* yacc ex3-3.yacc */
%union {
  Node* np;
  int i;
}
%type <np> stlist stat expr prim var
%token NUM;
%token IDENT;
%token READ;
%token PRINT;
%%
prog   : IDENT '{' stlist '}'       { emit_asm($3); return 0; }
       ;
stlist :                            { $$ = NULL; }
       | stlist stat                { $$ = createNode(T_STLIST, $1, $2); }
       ;
stat   : var '=' expr ';'           { $$ = createNode(T_ASSIGN, $1, $3); }
       | READ var ';'               { $$ = createNode(T_READ, $2, NULL); }
       | PRINT expr ';'             { $$ = createNode(T_PRINT, $2, NULL); }
       ;
expr   : prim              { $$ = $1; }
       | expr '+' prim     { $$ = createNode(T_ADD, $1, $3); }
       | expr '-' prim     { $$ = createNode(T_SUB, $1, $3); }
       ;
prim   : NUM               { $$ = createNode(T_NUM, atoi(yytext), NULL); }
       | var               { $$ = createNode(T_VAR, $1, NULL); }
       | '(' expr ')'      { $$ = $2; }
       ;
var    : IDENT             { $$ = lookup(yytext); }
       ;

Yaccで，`emit_c`だったところが`emit_asm`になっているだけです．

Cプログラムは以下のようになります．

In [ ]:
/* c ex3-3.c */
#include <stdio.h>
struct stab {
  int val;
  char name[20];
} stab[100];
int stabuse = 0;

extern char *yytext;

#define T_STLIST 1
#define T_ASSIGN 2
#define T_READ   3
#define T_PRINT  4
#define T_ADD    5
#define T_SUB    6
#define T_NUM    7
#define T_VAR    8

typedef struct Node {
    int node_type;
    struct Node* left;
    struct Node* right;
} Node;


// 引数にとる文字列が表にあれば見つかった位置を返して，なければ追加する関数
int lookup(char *s) {
  int i;
  // 表の使っているところを順に見ていって
  for (i = 0; i < stabuse; ++i) {
    if (strcmp(stab[i].name, s) == 0) {
      // 見つかった場合はその位置を返す
      return i;
    }
  }
  if (stabuse >= 99) {
    // 変数の上限の個数を超えてしまった
    printf("table overflow,\n");
    exit(1);
  }

  // 表になかったので追加する
  strcpy(stab[stabuse].name, s);
  return stabuse++;  // 追加した位置を返す
}

// 新しいノードを作る関数
Node* createNode(int node_type, Node* left, Node* right) {
    Node* newNode = (Node*)malloc(sizeof(Node));
    newNode->node_type = node_type;
    newNode->left = left;
    newNode->right = right;
    return newNode;
}

void _emit_asm(Node* node) {
  if (node == NULL) {
    return;
  }
  switch (node->node_type) {
    case T_STLIST:
      if (node->left != NULL) { _emit_asm(node->left); }
      _emit_asm(node->right);
      break;
    case T_ASSIGN:
      _emit_asm(node->right);
      printf("  popq %d(%%rbp)\n", -((int)node->left + 1) * 8);
      break;
    case T_READ:
      printf("  movq $.Lprompt, %%rdi\n");
      printf("  movq $0, %%rax\n");
      printf("  call printf\n");
      printf("  leaq %d(%%rbp), %%rsi\n", -((int)node->left + 1) * 8);
      printf("  movq $.Lread, %%rdi\n");
      printf("  movq $0, %%rax\n");
      printf("  call scanf\n");
      break;
    case T_PRINT:
      _emit_asm(node->left);
      printf("  popq %%rsi\n");
      printf("  movq $.Lprint, %%rdi\n");
      printf("  movq $0, %%rax\n");
      printf("  call printf\n");
      break;
    case T_ADD:
      _emit_asm(node->left);
      _emit_asm(node->right);
      printf("  popq %%rdx\n");
      printf("  popq %%rax\n");
      printf("  addq %%rdx, %%rax\n");
      printf("  pushq %%rax\n");
      break;
    case T_SUB:
      _emit_asm(node->left);
      _emit_asm(node->right);
      printf("  popq %%rdx\n");
      printf("  popq %%rax\n");
      printf("  subq %%rdx, %%rax\n");
      printf("  pushq %%rax\n");
      break;
    case T_NUM:
      printf("  pushq $%d\n", node->left);
      break;
    case T_VAR:
      printf("  pushq %d(%%rbp)\n", -((int)node->left + 1) * 8);
      break;
  }
}

void emit_asm(Node* node) {
  int stk;
  printf("     .section .rodata\n");
  printf(".Lprompt: .string \"> \"\n");
  printf(".Lread: .string \"%%ld\"\n");
  printf(".Lprint: .string \"%%ld\\n\"\n");
  printf("     .text\n");
  printf(".globl main\n");
  printf("main:\n");
  printf("     pushq %%rbp\n");
  printf("     movq %%rsp, %%rbp\n");
  stk = (8 * stabuse + 15) / 16;
  stk *= 16;
  printf("     subq $%d, %%rsp\n", stk);
  _emit_asm(node);
  printf("     leave\n");
  printf("     ret\n\n");
}

#include "y.tab.c"
#include "lex.yy.c"

int main() {
  yyparse();
  return 0;
}

`emit_asm()`では，アセンブリ言語に必要なおきまりのコードを出力し，プログラム本体のコードを生成する`_emit_asm()`を呼び出します．
今の言語には，`read`文と`print`文で入出力を扱いますが，これはCの標準ライブラリにある`scanf()`と`printf()`を使います．これらの関数は，引数として浮動小数点数を受けとる可能性があるため，これらの関数を呼び出すときには `rax` に `0` を設定しておく必要があります．

ラベル `.Lprompt`は，`printf()`で，`.Lread`は`scanf()`で，`.Lprint`は `printf()`でそれぞれ使うための制御文字列「`"> "`」，「`"%ld"`」「`"%ld\n"`」を入れた場所（アドレス）を参照するためのラベルです（符号付き64ビット長の値を読み書きするので，`%ld`）．ディレクティブ「`.string 文字列`」は，メモリを「文字列」で初期化するためです（Cと同様「文字列」はnull終端される）． 

`_emit_asm()`の処理も出力する言語がCからアセンブリ言語になっているだけで，`_emit_c()`と同様です．
出力するときに，実行時の途中結果を渡すかは，勝手に決めればよいので，**式の計算結果はスタックトップに置いてあるもの**としています．

- T_ASSIGN : 右辺を計算するコードを生成し，（結果はスタックトップにあるので）スタックから変数の場所にポップします．
- T_STLIST : 左側に文の並びがあれば，まずそのコードを出力し，続いて右側にある文のコードを出力します．
- T_READ : まずは，プロンプト「`> `」を出力します．それには，書き出すべき文字列のアドレスを（`printf()`の第1引数である）`%rdi`に入れて `printf()` を呼び出します．次に，変数のアドレス「`%rbp - (変数番号 + 1) * 8`」を（`scanf()`の第2引数である）レジスタ`%rsi`に，読み取り書式の文字列「`"%ld"`」のアドレス（第1引数である）レジスタ `%rdi` に入れて `scanf()` を呼び出します．変数のアドレスは，`%rbp - 8`，`%rbp - 16`であることに注意．C言語で

  ```
    printf("> ");
    scanf("%ld", &変数);
  ```
  と書いたコードに相当します．

- T_PRINT : まず，`_emit_asm()` で式のコードを出力します．（それを実行したらスタックトップに式の結果が置かれているはずなので）スタックトップを（`printf()`の第2引数である）レジスタ`%rsi`にポップします．出力書式の文字列「`"%ld\n"`」のアドレスを（第1引数である）レジスタ `%rdi` に入れて，`printf()`を呼び出します．なお`printf()`には，（ここでは使っていないが）引数として浮動小数点数が渡る可能性があるため，レジスタ`%rax`にゼロを入れておく必要があります．C言語で

  ```
    printf("%ld\n", 式);
  ```
  と書いたコードに相当します．

- T_ADD，T_SUB : 左の項を計算するコードを生成し，ついで右の項を計算するコードを生成します．これでスタックは上から，右の項の結果，左の項の結果となっているので，右の項の値をレジスタ`%rdx`にポップし，ついで左の項の値を`%rax`にポップし，「`%rax←%rax+%rdx`」を計算して，その結果をプッシュします．
- T_NUM : 上で決めたことにより，値をスタックにプッシュします．
- T_VAR : 同様に，変数の値をスタックにプッシュします．

それでは，実行してみましょう．C言語へのコンパイラのときには，出力されたCのコードを手動でコンパイルしてみましょう，と説明しましたが，アセンブリ言語を実行するときにはどうすれば良いか説明していませんでした．

アセンブリ言語のコードから，実行形式を作るには，`cc`コマンドを使います．
```
% cc ファイル名.s
```

`cc`コマンドは，
1. サフィックスが `.c` のファイルは，Cソースだと思ってアセンブリ語に翻訳し，翻訳結果をアセンブルする．
2. サフィックスが `.s` のファイルは，直接アセンブルする．
3. いずれの場合も，アセンブル結果の `.o` ファイルを C の標準ライブラリとリンクして `a.out` ファイルを作る．

ということを行います．

今回のこのノートブックでは，直前に作成された `a.out` を自前のコンパイラだと思って（**a.outを作り忘れないように**），自分言語のプログラムを入力として受けとる `uecc` というノートブックのコマンドを作成してあります．ですので，今回は，上で `emit_asm()` を実装したCプログラムを正しく実行できていればコンパイラの`a.out`が生成できているはずで，以下のコードを実行すると，`a.out`が作成できます．

In [ ]:
/* uecc ex3-3.uec */
main {
  read x;
  y = x + 1;
  print y;
}

確認のためにアセンブリ言語のコードを出力しています．このコードが `.uec.s` ファイルに保存されて，`cc`コマンドで `a.out` が作成されました．

1つ数字を読み込んで，その数に1を足すだけのプログラムですが，以下のように実行できます．

In [ ]:
/* a.out */
8

以上のように，C言語へコンパイルしていたのと同様に，アセンブリ言語へもコンパイルできました．
C言語のときと比べると，
- 実行形式を考える必要がある（スタックの使い方など）
- アセンブリ言語の決まりを意識する必要がある

という点はありますが，コンパイラにおけるコード生成について理解できたものと思います．

# 練習問題

提出は任意です．

順番にやる必要はなく，できるものから取り組んでください．

## 提出方法

以下の問題について，この3.ipynbファイルを編集し，完成した自分が編集した .ipynb ファイルをGoogle Classroomの該当課題に提出してください．（もしこのノートブックをファイルに保存したいときには，メニューの「file」→「Download」で .ipynb ファイルをダウンロード出来るので，それを提出してください．）

- 適宜コメントを入れてください．
- 提出する際のファイル名は，「**`学籍番号.ipynb`**」としてください．

## 問題1

C言語へのコンパイラで，`while`文に対応してください．
前回の練習問題を解いた人は，複合文や `<=`や論理演算などに対応したものでもよいです．

ヒント: LexとYaccは前回のままでいけるはず（`while`だけではなくて，`<`や`>`や`++`を追加するのも忘れないこと）

In [ ]:
/* lex p3-1.lex */
ここに書く

In [ ]:
/* yacc p3-1.yacc */
ここに書く

In [ ]:
/* c p3-1.c */
ここに書く

In [ ]:
/* a.out */
自分の書いた電卓のプログラムが正しく動作をしていることを確認できる入力を考えて書く

## 問題2

アセンブリ言語へのコンパイラで，`while`文に対応してください．
いきなり実行すると難しいと思うので，簡単な例を使って，出てきたアセンブリ言語のコードをちゃんと読んで想定しているようになっているか見てみるとよいと思います．


- ヒント: ループの条件の飛び先を表わす**ラベル**を適宜付けていく
    つまり，
  
      ```
      ラベル1:
        条件に対応するコードを生成
        条件が不成立ならばラベル2へ分岐
            条件に合致する場合に実行する命令列（whileの本体）
            ラベル1へ
      ラベル2:
      ```

    というような命令列にすればよいですね．
- ヒント: ラベル番号を管理できるように

     ```
     static int labelno = 1;
     ```
     
     というような定義を `emit_asm`内に追加しましょう．
- ヒント: `cmp`命令は，`cmp src, dst` の形式で，`src - dst` の結果をコンディションコードレジスタに暗黙的に入れてくれます．
     「x64アセンブリ言語の説明」をよく読むこと．
- ヒント: `cmp`命令の直後で，`jge ラベル` のような命令でコンディションコードレジスタの結果からジャンプできます．
- ヒント（難しい箇所なので重要）: `while ( cond ) 式` のような構文なので，`cond`に対応するコードは `<` や `>` を表わす `T_LT` や `T_GT` の抽象構文木のノードになっているはずです．なので，`T_LT`や`T_GT`でコードを出力する箇所（`_emit_asm`の`switch`の分岐）では，`jge`や`jle`だけを出力しておいて（ラベルは表示しない），`T_WHILE`の箇所でラベルを出力するようにしましょう． （単独の式として出てくる `x = 1 < 5`のようなものは，今の言語にはないのでこれでよい）
- ヒント: `T_WHILE`の箇所でラベルを出力する際には，適宜`labelno++`して，ラベルがちゃんと被らないようにしましょう．

In [ ]:
/* lex p3-2.lex */
ここに書く

In [ ]:
/* yacc p3-.yacc */
ここに書く

In [ ]:
/* c p3-.c */
ここに書く

In [ ]:
/* a.out */
自分の書いた電卓のプログラムが正しく動作をしていることを確認できる入力を考えて書く